# partie1 catégorie sociologue

In [1]:
import csv
import numpy as np
import pandas as pd

In [2]:
df_age = pd.read_csv('age.csv')
df_age.index = list(df_age.id_age)
df_age = df_age[['text']]
df_age.columns = ['age']

df_csp = pd.read_csv('categorie_socio.csv')
df_csp.index = list(df_csp.id_csp)
df_csp = df_csp[['text']]
df_csp.columns = ['csp']

df_gen = pd.read_csv('genre.csv')
df_gen.index = list(df_gen.id_genre)
df_gen = df_gen[['text']]
df_gen.columns = ['gen']

df_par = pd.read_csv('participant.csv')
df_par.index = list(df_par.id_participant)
list_participant = list(df_par.id_participant)
df_par = df_par[['id_age', 'id_csp', 'id_genre']]
df_par.columns = ['id_age', 'id_csp', 'id_gen']

df1 = df_par.join(df_age, on = 'id_age')
df2 = df1.join(df_csp, on = 'id_csp')
df3 = df2.join(df_gen, on = 'id_gen')
df4 = df3[['age', 'csp', 'gen']]

df4.head()

,age,csp,gen
30,18-25 ans,"Élèves, étudiants",Femme
31,18-25 ans,"Élèves, étudiants",Homme
32,18-25 ans,"Élèves, étudiants",Homme
33,18-25 ans,"Élèves, étudiants",Femme
34,18-25 ans,"Élèves, étudiants",Femme


# partie2 durée de chaque époque

In [3]:
from datetime import datetime

In [4]:
df_temps = pd.read_csv("participant.csv",sep=',') 
df_temps.index = list_participant
df_temps = df_temps[['t_drag1', 't_drag2', 't_note1', 't_not2', 't_csp']]

df_temps['t_drag1'] = pd.to_datetime(df_temps['t_drag1'],infer_datetime_format=True)
df_temps['t_drag2'] = pd.to_datetime(df_temps['t_drag2'],infer_datetime_format=True)
df_temps['t_note1'] = pd.to_datetime(df_temps['t_note1'],infer_datetime_format=True)
df_temps['t_not2'] = pd.to_datetime(df_temps['t_not2'],infer_datetime_format=True)
df_temps['t_csp'] = pd.to_datetime(df_temps['t_csp'],infer_datetime_format=True)

In [5]:
frames = []
column3 = ['td1', 'td2', 'td3', 'td4']
for i in range(0, len(list_participant)): 
    name = list_participant[i]
    a = list(df_temps.iloc[i])
    a = sorted(a)
    b = []
    for i in range(0, len(a) - 1): 
        b.append(a[i+1] - a[i])
    c = pd.DataFrame(b, index = column3, columns = [name]).T
    frames.append(c)
df5 = pd.concat(frames)
df5.head()

,td1,td2,td3,td4
30,00:01:20,00:00:39,00:00:45,00:00:26
31,00:00:58,00:00:45,00:01:05,00:00:26
32,00:01:24,00:00:54,00:00:49,00:00:41
33,00:01:33,00:00:29,00:01:19,00:00:26
34,00:02:07,00:00:35,00:02:18,00:00:34


# partie3 différence de repas et de note

In [6]:
#Choix de la distance :(là c'est edition)
Choix_distance = "matrice"
Choix_distance = "edition"

In [7]:
df_res = pd.read_csv("reponse.csv", header = None)
df_res.columns = ['id', 'repas', 'sets', 'temps', 'notes']
df_res.index = list(df_res['id'])
df_res = df_res[['repas', 'temps', 'notes']]
df_res.head()

,repas,temps,notes
30,A1P1D1,m1,5
30,A1P1D1-2,m9,5
30,A1P1D2,m5,4
30,A1P2D1,m8,5
30,A1P2D2,m4,4


In [8]:
df_rep = pd.read_csv("repas.csv", header = None)
df_rep.columns = ['repas', 'id', 'A', 'B', 'accompagnment', 'protein', 'dessert']
df_rep = df_rep[['repas', 'id', 'accompagnment', 'protein', 'dessert']]
df_rep.head()

,repas,id,accompagnment,protein,dessert
0,A1P1D1,1,1,1,1
1,A1P1D1-2,2,1,1,1
2,A1P1D2,3,1,1,2
3,A1P2D1,10,1,2,1
4,A1P2D2,9,1,2,2


In [9]:
# Choix_distance = "matrice"
Choix_distance = "edition"

def ddm(series1, series2): 
    if Choix_distance == "edition":
        distance1 = 0
        distance2 = 0
        for i in range(4,7): 
            distance1 += abs(int(series1[i]) - int(series2[i]))
        for i in range(11,14):
            distance2 += (abs(int(series1[i]) - int(series2[i])) >= 1)*1
    else :
        
        p = np.array([
            [0, 0, 1, 2, 3, 4, 5, 6, 7, 7], 
            [0, 0, 0, 2, 3, 4, 5, 6, 7, 7], 
            [1, 0, 0, 4, 6, 2, 7, 3, 5, 5], 
            [2, 2, 4, 0, 5, 1, 3, 7, 6, 6], 
            [3, 3, 6, 5, 0, 7, 2, 1, 4, 4], 
            [4, 4, 2, 1, 7, 0, 6, 5, 3, 3], 
            [5, 5, 7, 3, 2, 6, 0, 4, 1, 1], 
            [6, 6, 3, 7, 1, 5, 4, 0, 2, 2], 
            [7, 7, 5, 6, 4, 3, 1, 2, 0, 0], 
            [7, 7, 5, 6, 4, 3, 1, 2, 0, 0]
        ])
        v = np.array([
            [0, 7, 7, 5, 6, 4, 4, 1, 7, 6], 
            [7, 0, 4, 1, 2, 6, 7, 5, 2, 4], 
            [7, 4, 0, 4, 4, 5, 3, 7, 1, 2], 
            [5, 1, 4, 0, 4, 6, 7, 7, 4, 4], 
            [6, 2, 4, 4, 0, 7, 7, 3, 2, 1], 
            [4, 6, 5, 6, 7, 0, 2, 4, 7, 5], 
            [4, 7, 3, 7, 7, 2, 0, 4, 6, 5], 
            [1, 5, 7, 7, 3, 4, 4, 0, 5, 6], 
            [7, 2, 1, 4, 2, 7, 6, 5, 0, 4], 
            [6, 4, 2, 4, 1, 5, 5, 6, 4, 0]
        ])
        p=np.where(p==1, 20, p)
        p=np.where(p==3, 1, p)
        p=np.where(p==2, 3, p)
        p=np.where(p==20, 2, p)
        p=np.where(p==4, 60, p)
        p=np.where(p==6, 4, p)
        p=np.where(p==60, 6, p)
        v=np.where(v==1, 20, v)
        v=np.where(v==3, 1, v)
        v=np.where(v==2, 3, v)
        v=np.where(v==20, 2, v)
        v=np.where(v==4, 60, v)
        v=np.where(v==6, 4, v)
        v=np.where(v==60, 6, v)
        distance1 = p[int(series1[3])-1][int(series2[3])-1]
        distance2 = v[int(series1[10])-1][int(series2[10])-1]
        
    return distance1, distance2

In [10]:
def distance_note(ligne1, ligne2):
    
    prem = int(ligne1[2])
    sec = int(ligne2[2])
    var1 = (prem != sec)*(-1)**(prem>sec)

    prem = int(ligne1[9])
    sec = int(ligne2[9])
    var2 = (prem != sec)*(-1)**(prem>sec)
    return var1, var2

In [11]:
column = []
for i in range(1, 21): 
    column.append(str(i))
    
column2 = []
for i in range(0, 4): 
    for j in range(1, 18): 
        if i == 0: 
            if j <= 9: 
                column2.append('s_p_ms_' + str(j))
            elif j <= 13: 
                column2.append('s_p_mm_' + str(j - 9))
            else: 
                column2.append('s_p_ss_' + str(j - 13))
        elif i == 1: 
            if j <= 9: 
                column2.append('s_v_ms_' + str(j))
            elif j <= 13: 
                column2.append('s_v_mm_' + str(j - 9))
            else: 
                column2.append('s_v_ss_' + str(j - 13))
        elif i == 2: 
            if j <= 9: 
                column2.append('n_p_ms_' + str(j))
            elif j <= 13: 
                column2.append('n_p_mm_' + str(j - 9))
            else: 
                column2.append('n_p_ss_' + str(j - 13))
        else: 
            if j <= 9: 
                column2.append('n_v_ms_' + str(j))
            elif j <= 13: 
                column2.append('n_v_mm_' + str(j - 9))
            else: 
                column2.append('n_v_ss_' + str(j - 13))

In [12]:
frames = []
for i in range(0, len(list_participant)): 
    name = list_participant[i]
    df_participant = df_res[df_res.index == list_participant[i]]
    df_participant1 = pd.merge(df_participant, df_rep, on = 'repas')
    df_participant1.index = df_participant.index
    df_participant2 = pd.concat([df_participant1[0:10].sort_values(by = "temps"), 
                                 df_participant1[10:20].sort_values(by = "temps")], 
                                 axis = 1)
    df_participant2.columns = ['repas_p', 'temps_p', 'notes_p', 
                               'i_p', 'a_p', 'p_p', 'd_p', 
                               'repas_v', 'temps_v', 'notes_v', 
                               'i_v', 'a_v', 'p_v', 'd_v']
    df_participant3 = pd.DataFrame([
        df_participant2.iloc[0], 
        df_participant2.iloc[2], 
        df_participant2.iloc[4], 
        df_participant2.iloc[6], 
        df_participant2.iloc[8]
    ])
    df_participant4 = pd.DataFrame([
        df_participant2.iloc[1], 
        df_participant2.iloc[3], 
        df_participant2.iloc[5], 
        df_participant2.iloc[7], 
        df_participant2.iloc[9]
    ])
    df_participant5 = pd.concat([df_participant2, df_participant3, df_participant4])
    list1 = []
    list2 = []
    
    note_p = []
    note_v = []
    for i in range(0,20):
        if i < 19:
            list1.append(ddm(df_participant5.iloc[i], df_participant5.iloc[i+1])[0])
            list2.append(ddm(df_participant5.iloc[i], df_participant5.iloc[i+1])[1])
            
            #recuperation des notes
            note_p.append(distance_note(df_participant5.iloc[i], df_participant5.iloc[i+1])[0])
            note_v.append(distance_note(df_participant5.iloc[i], df_participant5.iloc[i+1])[1])

            
        else: 
            list1.append(ddm(df_participant5.iloc[i], df_participant5.iloc[0])[0])
            list2.append(ddm(df_participant5.iloc[i], df_participant5.iloc[0])[1])
            note_p.append(distance_note(df_participant5.iloc[i], df_participant5.iloc[0])[0])
            note_v.append(distance_note(df_participant5.iloc[i], df_participant5.iloc[0])[1])
            
            
    df_participant5['score_p'] = list1
    df_participant5['score_v'] = list2
    
    df_participant5['var_note_p'] = note_p
    df_participant5['var_note_v'] = note_v

    test = df_participant5.ix[:,[3,14,2,16,10,15,9,17]]
    
    test2 = test.T
    test2.columns = column
    test2 = test2.drop(['10', '15', '20'], axis = 1)
    
    result = test2.iloc[1].append(test2.iloc[5])
    result = result.append(test2.iloc[3])
    result = result.append(test2.iloc[7])
    result = pd.DataFrame(result, columns = [name]).T
    result.columns = column2
    frames.append(result)

H:\anacoda\lib\site-packages\ipykernel_launcher.py:57: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
H:\anacoda\lib\site-packages\ipykernel_launcher.py:57: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
H:\anacoda\lib\site-packages\ipykernel_launcher.py:57: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
H:\anacoda\lib\site-packages\ipykernel_launcher.py:57: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for position

H:\anacoda\lib\site-packages\ipykernel_launcher.py:57: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
H:\anacoda\lib\site-packages\ipykernel_launcher.py:57: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
H:\anacoda\lib\site-packages\ipykernel_launcher.py:57: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
H:\anacoda\lib\site-packages\ipykernel_launcher.py:57: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for position

H:\anacoda\lib\site-packages\ipykernel_launcher.py:57: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
H:\anacoda\lib\site-packages\ipykernel_launcher.py:57: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
H:\anacoda\lib\site-packages\ipykernel_launcher.py:57: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
H:\anacoda\lib\site-packages\ipykernel_launcher.py:57: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for position

H:\anacoda\lib\site-packages\ipykernel_launcher.py:57: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
H:\anacoda\lib\site-packages\ipykernel_launcher.py:57: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
H:\anacoda\lib\site-packages\ipykernel_launcher.py:57: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
H:\anacoda\lib\site-packages\ipykernel_launcher.py:57: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for position

H:\anacoda\lib\site-packages\ipykernel_launcher.py:57: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
H:\anacoda\lib\site-packages\ipykernel_launcher.py:57: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
H:\anacoda\lib\site-packages\ipykernel_launcher.py:57: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
H:\anacoda\lib\site-packages\ipykernel_launcher.py:57: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for position

...

H:\anacoda\lib\site-packages\ipykernel_launcher.py:57: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
H:\anacoda\lib\site-packages\ipykernel_launcher.py:57: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
H:\anacoda\lib\site-packages\ipykernel_launcher.py:57: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
H:\anacoda\lib\site-packages\ipykernel_launcher.py:57: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for position

H:\anacoda\lib\site-packages\ipykernel_launcher.py:57: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
H:\anacoda\lib\site-packages\ipykernel_launcher.py:57: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
H:\anacoda\lib\site-packages\ipykernel_launcher.py:57: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
H:\anacoda\lib\site-packages\ipykernel_launcher.py:57: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for position

H:\anacoda\lib\site-packages\ipykernel_launcher.py:57: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
H:\anacoda\lib\site-packages\ipykernel_launcher.py:57: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
H:\anacoda\lib\site-packages\ipykernel_launcher.py:57: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
H:\anacoda\lib\site-packages\ipykernel_launcher.py:57: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for position

H:\anacoda\lib\site-packages\ipykernel_launcher.py:57: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
H:\anacoda\lib\site-packages\ipykernel_launcher.py:57: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
H:\anacoda\lib\site-packages\ipykernel_launcher.py:57: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
H:\anacoda\lib\site-packages\ipykernel_launcher.py:57: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for position

H:\anacoda\lib\site-packages\ipykernel_launcher.py:57: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
H:\anacoda\lib\site-packages\ipykernel_launcher.py:57: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
H:\anacoda\lib\site-packages\ipykernel_launcher.py:57: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
H:\anacoda\lib\site-packages\ipykernel_launcher.py:57: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for position

In [13]:
df6 = pd.concat(frames)
df6

,s_p_ms_1,s_p_ms_2,s_p_ms_3,s_p_ms_4,s_p_ms_5,s_p_ms_6,s_p_ms_7,s_p_ms_8,s_p_ms_9,s_p_mm_1,...,n_v_ms_8,n_v_ms_9,n_v_mm_1,n_v_mm_2,n_v_mm_3,n_v_mm_4,n_v_ss_1,n_v_ss_2,n_v_ss_3,n_v_ss_4
30,3,0,1,2,1,3,1,2,1,3,...,-1,1,-1,1,0,-1,-1,1,1,-1
31,2,2,1,2,2,2,1,1,1,2,...,-1,1,0,0,1,-1,1,0,1,0
32,3,0,1,3,1,3,1,3,1,3,...,1,1,-1,1,-1,1,1,-1,-1,1
33,2,1,1,3,2,3,2,3,2,3,...,1,1,0,-1,1,1,1,-1,0,1
34,1,2,2,2,2,3,2,2,2,3,...,0,-1,0,0,-1,1,0,1,1,-1
35,3,0,2,3,1,3,2,3,1,3,...,1,0,-1,0,1,0,0,1,-1,1
36,1,1,3,2,3,3,1,2,3,2,...,-1,1,1,-1,1,-1,-1,1,1,-1
37,3,0,1,2,2,1,2,3,1,3,...,1,-1,0,-1,0,0,1,0,-1,0
38,3,0,2,3,1,3,2,3,1,3,...,-1,1,1,-1,-1,0,-1,0,1,-1
39,2,1,1,2,1,2,2,2,1,3,...,1,1,0,1,-1,0,0,-1,-1,1


In [14]:
tableau_total = df6.copy()

for i in range(3, -1, -1): 
    tableau_total.insert(0, str(df5.columns[i]), df5[str(df5.columns[i])])
    
    
for i in range(2, -1, -1): 
    tableau_total.insert(0, str(df4.columns[i]), df4[str(df4.columns[i])])
    

tableau_total.head()

,age,csp,gen,td1,td2,td3,td4,s_p_ms_1,s_p_ms_2,s_p_ms_3,...,n_v_ms_8,n_v_ms_9,n_v_mm_1,n_v_mm_2,n_v_mm_3,n_v_mm_4,n_v_ss_1,n_v_ss_2,n_v_ss_3,n_v_ss_4
30,18-25 ans,"Élèves, étudiants",Femme,00:01:20,00:00:39,00:00:45,00:00:26,3,0,1,...,-1,1,-1,1,0,-1,-1,1,1,-1
31,18-25 ans,"Élèves, étudiants",Homme,00:00:58,00:00:45,00:01:05,00:00:26,2,2,1,...,-1,1,0,0,1,-1,1,0,1,0
32,18-25 ans,"Élèves, étudiants",Homme,00:01:24,00:00:54,00:00:49,00:00:41,3,0,1,...,1,1,-1,1,-1,1,1,-1,-1,1
33,18-25 ans,"Élèves, étudiants",Femme,00:01:33,00:00:29,00:01:19,00:00:26,2,1,1,...,1,1,0,-1,1,1,1,-1,0,1
34,18-25 ans,"Élèves, étudiants",Femme,00:02:07,00:00:35,00:02:18,00:00:34,1,2,2,...,0,-1,0,0,-1,1,0,1,1,-1


In [15]:
tableau_total.to_csv('tableau_total.csv')